In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

from surprise.model_selection import cross_validate

In [2]:
movie = pd.read_csv('movies_metadata.csv')

C:\Users\YOON HYE YEONG\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [56]:
movies = pd.read_csv('movies_metadata.csv')
movies.columns

C:\Users\YOON HYE YEONG\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [48]:
movie

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","[{'id': 10703, 'name': 'tragic love'}]"
46624,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46625,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",[]
46626,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,[],Released,NaN,Satan Triumphant,False,0.0,0.0,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[]


In [50]:
movie.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [4]:
movie['genres'] = movie['genres'].fillna('[]').apply(literal_eval).apply(lambda x : [i['name'] for i in x] if isinstance(x, list) else [])

In [5]:
link = pd.read_csv("links_small.csv")

In [6]:
link.head(3)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [7]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [8]:
print(movie.index[(movie['id'] == '2012-09-29')].tolist())

[29503]


In [9]:
#id 칼럼 object -> int

movie = movie.drop([19730, 29503, 35587])
movie = movie.astype({'id' : 'int'})

In [10]:
data = movie[movie['id'].isin(link['movieId'])]

In [11]:
data.shape

(2840, 24)

# overview와 tagline을 이용하여 추천 시스템 개발

In [12]:
data['tagline'] = data['tagline'].fillna('')
data['description'] = data['overview'] + data['tagline']
data['description'] = data['description'].fillna('')

<ipython-input-12-db52973e84b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tagline'] = data['tagline'].fillna('')
<ipython-input-12-db52973e84b4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['description'] = data['overview'] + data['tagline']
<ipython-input-12-db52973e84b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [13]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, 
                    stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['description'])

In [14]:
tfidf_matrix.shape

(2840, 95108)

In [15]:
#cosine similarity 계산

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
cosine_sim.shape

(2840, 2840)

In [17]:
data = data.reset_index()
titles = data['title']
indices = pd.Series(data.index, index=data['title'])

In [18]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [19]:
get_recommendations('The Godfather').head(10)

230         The Godfather: Part II
2525             Gang War in Milan
1144               Queen of Hearts
1825    The Cave of the Yellow Dog
1421           Running Out of Time
399        The Godfather: Part III
948                        8 Women
1268                 Bright Future
747              Jaws: The Revenge
2184             Short Sharp Shock
Name: title, dtype: object

In [20]:
get_recommendations('The Dark Knight').head(10)

24           Batman Forever
293          Batman Returns
111                  Batman
626                     JFK
1529          Batman Begins
1295        To End All Wars
322          Batman & Robin
2583    The Eleventh Victim
1615             Cul-de-sac
1831                 Taxi 4
Name: title, dtype: object

# 장르, 출연진, 제작진을 고려 요소로 채택

In [21]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

In [52]:
keywords.head(3)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


In [22]:
keywords = keywords.astype({'id' : 'int'})
credits = credits.astype({'id' : 'int'})
movie = movie.astype({'id' : 'int'})

In [23]:
movie.shape

(45463, 24)

In [24]:
movie = movie.merge(credits, on = 'id')
movie = movie.merge(keywords, on = 'id')

In [25]:
data = movie[movie['id'].isin(link['movieId'])]
data.shape

(2858, 27)

In [26]:
data.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...","[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."
14,False,NaN,98000000,"[Action, Adventure]",NaN,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,5.7,137.0,"[{'cast_id': 1, 'character': 'Morgan Adams', '...","[{'credit_id': '52fe42f4c3a36847f802f69f', 'de...","[{'id': 911, 'name': 'exotic island'}, {'id': ..."


In [46]:
data['crew'][0]

[{'credit_id': '52fe4292c3a36847f802916d',
  'department': 'Directing',
  'gender': 2,
  'id': 638,
  'job': 'Director',
  'name': 'Michael Mann',
  'profile_path': '/nKmUpRpuQIsYubR7vIxVKhkbaTW.jpg'},
 {'credit_id': '52fe4292c3a36847f8029173',
  'department': 'Writing',
  'gender': 2,
  'id': 638,
  'job': 'Screenplay',
  'name': 'Michael Mann',
  'profile_path': '/nKmUpRpuQIsYubR7vIxVKhkbaTW.jpg'},
 {'credit_id': '52fe4292c3a36847f8029179',
  'department': 'Production',
  'gender': 2,
  'id': 1254,
  'job': 'Producer',
  'name': 'Art Linson',
  'profile_path': '/dEtVivCXxQBtIzmJcUNupT1AB4H.jpg'},
 {'credit_id': '52fe4292c3a36847f802917f',
  'department': 'Production',
  'gender': 2,
  'id': 638,
  'job': 'Producer',
  'name': 'Michael Mann',
  'profile_path': '/nKmUpRpuQIsYubR7vIxVKhkbaTW.jpg'},
 {'credit_id': '52fe4292c3a36847f8029185',
  'department': 'Sound',
  'gender': 2,
  'id': 5581,
  'job': 'Original Music Composer',
  'name': 'Elliot Goldenthal',
  'profile_path': '/mr1rr5b

In [28]:

data['cast'] = data['cast'].apply(literal_eval)
data['crew'] = data['crew'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

<ipython-input-28-806bfeb60a00>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cast'] = data['cast'].apply(literal_eval)
<ipython-input-28-806bfeb60a00>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['crew'] = data['crew'].apply(literal_eval)
<ipython-input-28-806bfeb60a00>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [29]:
data['cast_size'] = data['cast'].apply(lambda x : len(x))
data['crew_size'] = data['crew'].apply(lambda x : len(x))

<ipython-input-29-9b1245814ff1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cast_size'] = data['cast'].apply(lambda x : len(x))
<ipython-input-29-9b1245814ff1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['crew_size'] = data['crew'].apply(lambda x : len(x))


In [30]:
def get_director(x) : 
    for i in x :
        if i['job'] == 'Director' :
            return i['name']
    return np.nan

In [31]:
data['director'] = data['crew'].apply(get_director)

<ipython-input-31-88b517b8de11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['director'] = data['crew'].apply(get_director)


In [32]:
data['cast'] = data['cast'].apply(lambda x : [i['name'] for i in
            x] if isinstance(x, list) else [])

data['cast'] = data['cast'].apply(lambda x : x[:3] if len(x) >= 3
                               else x)

<ipython-input-32-61063952ecf1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cast'] = data['cast'].apply(lambda x : [i['name'] for i in
<ipython-input-32-61063952ecf1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cast'] = data['cast'].apply(lambda x : x[:3] if len(x) >= 3


In [33]:
data['keywords'] = data['keywords'].apply(lambda x : [i['name'] for i in
            x] if isinstance(x, list) else [])

<ipython-input-33-f2c0725c8b82>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywords'] = data['keywords'].apply(lambda x : [i['name'] for i in


In [34]:
data['director'] = data['director'].astype('str').apply(lambda x: str.lower(x.replace(" ","")))
data['director'] = data['director'].apply(lambda x: [x, x, x])

<ipython-input-34-67c4f321be38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['director'] = data['director'].astype('str').apply(lambda x: str.lower(x.replace(" ","")))
<ipython-input-34-67c4f321be38>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['director'] = data['director'].apply(lambda x: [x, x, x])


In [35]:
s = data.apply(lambda x: pd.Series(x['keywords']), axis = 1).stack().reset_index(level = 1, drop=True)
s.name = 'keyword'

<ipython-input-35-9ef55058db6e>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = data.apply(lambda x: pd.Series(x['keywords']), axis = 1).stack().reset_index(level = 1, drop=True)


In [36]:
s = s.value_counts()
s[:5]

independent film    165
woman director      149
murder              136
dying and death      88
violence             86
Name: keyword, dtype: int64

In [37]:
s = s[s>1]
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [38]:
def filter_keywords(x) :
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [39]:
data['keywords'] = data['keywords'].apply(filter_keywords)
data['keywords'] = data['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
data['keywords'] = data['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

<ipython-input-39-8a78a51966c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywords'] = data['keywords'].apply(filter_keywords)
<ipython-input-39-8a78a51966c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['keywords'] = data['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
<ipython-input-39-8a78a51966c7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [40]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast,crew,keywords,cast_size,crew_size,director
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,Heat,False,7.7,1886.0,"[Al Pacino, Robert De Niro, Val Kilmer]","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[robberi, detect, bank, obsess, chase, shoot, ...",65,71,"[michaelmann, michaelmann, michaelmann]"
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,GoldenEye,False,6.6,1194.0,"[Pierce Brosnan, Sean Bean, Izabella Scorupco]","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...","[cuba, falselyaccus, secretident, computerviru...",20,46,"[martincampbell, martincampbell, martincampbell]"
14,False,NaN,98000000,"[Action, Adventure]",NaN,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,Cutthroat Island,False,5.7,137.0,"[Geena Davis, Matthew Modine, Frank Langella]","[{'credit_id': '52fe42f4c3a36847f802f69f', 'de...","[exoticisland, treasur, map, ship, pirat]",31,16,"[rennyharlin, rennyharlin, rennyharlin]"
15,False,NaN,52000000,"[Drama, Crime]",NaN,524,tt0112641,en,Casino,The life of the gambling paradise – Las Vegas ...,...,Casino,False,7.8,1343.0,"[Robert De Niro, Sharon Stone, Joe Pesci]","[{'credit_id': '52fe424dc3a36847f80139cd', 'de...","[poker, drugabus, 1970s, overdos, illegalprost...",45,10,"[martinscorsese, martinscorsese, martinscorsese]"
16,False,NaN,16500000,"[Drama, Romance]",NaN,4584,tt0114388,en,Sense and Sensibility,"Rich Mr. Dashwood dies, leaving his second wif...",...,Sense and Sensibility,False,7.2,364.0,"[Kate Winslet, Emma Thompson, Hugh Grant]","[{'credit_id': '52fe43cec3a36847f807101f', 'de...","[bowl, basedonnovel, servant, countrylif, inhe...",17,8,"[anglee, anglee, anglee]"


In [41]:
data['soup'] = data['keywords'] + data['cast'] + data['director'] + data['genres']
data['soup'] = data['soup'].apply(lambda x: ' '.join(x))

<ipython-input-41-ff7289e6d7ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['soup'] = data['keywords'] + data['cast'] + data['director'] + data['genres']
<ipython-input-41-ff7289e6d7ed>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['soup'] = data['soup'].apply(lambda x: ' '.join(x))


In [42]:
count = CountVectorizer(analyzer = 'word', ngram_range = (1, 2),
                        min_df = 0,stop_words='english')

count_matrix = count.fit_transform(data['soup'])

In [43]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [44]:
data = data.reset_index()
titles = data['title']
indices = pd.Series(data.index, index=data['title'])

In [45]:
get_recommendations('The Dark Knight').head(10)

1536     Batman Begins
1716      The Prestige
920           Insomnia
759            Memento
293     Batman Returns
2070       Harry Brown
322     Batman & Robin
111             Batman
716         Get Carter
24      Batman Forever
Name: title, dtype: object